In [ ]:
from grid_optimizer import GridOptimizer
from tqdm import tqdm
from data import Data
from plotting import CandlePlot
import pandas as pd

data_path = '../data/'
out_path = 'D:/Trading/ml4t-data/grid/'
instruments = "../data/instruments.json"

In [ ]:
dummyrun = False
checkpoint=0
counter=50000
start=None  
end=None
records=['events']
tickers=['EUR_USD']
frequency=['M5']
init_bal=[1000]
init_trade_size=[100]
grid_pips=[10, 20]
sl_grid_count=[5] #, 3, 5, 10, 20]
stop_loss_type=['grid_count_max_unrealised_pnl']
margin_sl_percent=[0.80]
sizing=['dynamic', 'static']
cash_out_factor=[None]
cover_stopped_loss = [False]
martingale_sizing = [False]
max_unrealised_pnl = [0.10]
inputs_file='inputs.30c.csv'

In [ ]:
optim = GridOptimizer(
    checkpoint=checkpoint,
    counter=counter,
    start=start,
    end=end,
    records=records,
    tickers=tickers,
    frequency=frequency,
    init_bal=init_bal,      
    init_trade_size=init_trade_size,
    grid_pips=grid_pips,
    sl_grid_count=sl_grid_count,
    stop_loss_type=stop_loss_type,
    margin_sl_percent=margin_sl_percent,
    sizing=sizing,
    cash_out_factor=cash_out_factor,
    cover_stopped_loss=cover_stopped_loss,
    martigale_sizing=martingale_sizing,
    max_unrealised_pnl=max_unrealised_pnl,
    data_path=data_path,
    instruments=instruments,
    out_path=out_path,
    inputs_file=inputs_file,
    dummyrun=dummyrun
)

In [ ]:
tk = tickers[0]
f = frequency[0]
df = optim.read_data(tk, f).iloc[start:end]

In [ ]:
df['dt'] = df.time.dt.date
df['yearmonth'] = df.time.dt.year.astype(str) + '-' + df.time.dt.month.astype(str)
df

In [ ]:
df.groupby(by='dt').count()['time'].mean()

In [ ]:
df.groupby(by='yearmonth').count()['time'].mean()

In [ ]:
6190.65625 / 238.77179590196866

In [ ]:
df['newdate'] = df.dt != df.dt.shift(1)
df.iloc[280:300]

In [ ]:
df['newmonth'] = df.yearmonth != df.yearmonth.shift(1)

In [ ]:
df2 = df.iloc[:].copy()

In [ ]:
# set first newdate / newmonth to False
df2.iloc[0, -1] = False
df2.iloc[0, -2] = False
df2

In [ ]:
# df2['mean_c'] = np.where(df.newdate == True, df2[df2.dt], None)

In [ ]:
d =Data(
    source=df2.copy(),
    ticker=tickers[0],
    cols=['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'dt', 'newdate', 'yearmonth', 'newmonth'],
    instruments=instruments
)

In [ ]:
add_cols = dict(
    dayhigh=float,
    daylow=float,
    daymid=float,
    daymean=float,
    daymedian=float,
    day1stdup=float,
    day1stddown=float,
    day2stdup=float,
    day2stddown=float,
    day3stdup=float,
    day3stddown=float,
    monthhigh=float,
    monthlow=float,
    monthmid=float,
    monthmean=float,
    monthmedian=float,
    month1stdup=float,
    month1stddown=float,
    month2stdup=float,
    month2stddown=float,
    month3stdup=float,
    month3stddown=float
)

d.prepare_fast_data(name='test', start=0, end=d.datalen, add_cols=add_cols)

In [ ]:
d

In [ ]:
for i in tqdm(range(d.fdatalen), desc=" Processing... "):
    if d.fdata('newdate', i) == True:
        prevday_df = d.df['test'][d.df['test'].dt == d.fdata('dt', i-1)]
        d.update_fdata('dayhigh', i, prevday_df.mid_c.max())
        d.update_fdata('daylow', i, prevday_df.mid_c.min())
        d.update_fdata('daymid', i, (prevday_df.mid_c.max() + prevday_df.mid_c.min()) / 2)
        d.update_fdata('daymean', i, prevday_df.mid_c.mean())
        d.update_fdata('daymedian', i, prevday_df.mid_c.median())
        std = prevday_df.mid_c.std()
        d.update_fdata('day1stdup', i, prevday_df.mid_c.mean() + std)
        d.update_fdata('day1stddown', i, prevday_df.mid_c.mean() - std)
        d.update_fdata('day2stdup', i, prevday_df.mid_c.mean() + 2 * std)
        d.update_fdata('day2stddown', i, prevday_df.mid_c.mean() - 2 * std)
        d.update_fdata('day3stdup', i, prevday_df.mid_c.mean() + 3 * std)
        d.update_fdata('day3stddown', i, prevday_df.mid_c.mean() - 3 * std)
    if d.fdata('newmonth', i) == True:
        prevmonth_df = d.df['test'][d.df['test'].yearmonth == d.fdata('yearmonth', i-1)]
        d.update_fdata('monthhigh', i, prevmonth_df.mid_c.max())
        d.update_fdata('monthlow', i, prevmonth_df.mid_c.min())
        d.update_fdata('monthmid', i, (prevmonth_df.mid_c.max() + prevmonth_df.mid_c.min()) / 2)
        d.update_fdata('monthmean', i, prevmonth_df.mid_c.mean())
        d.update_fdata('monthmedian', i, prevmonth_df.mid_c.median())
        std = prevmonth_df.mid_c.std()
        d.update_fdata('month1stdup', i, prevmonth_df.mid_c.mean() + std)
        d.update_fdata('month1stddown', i, prevmonth_df.mid_c.mean() - std)
        d.update_fdata('month2stdup', i, prevmonth_df.mid_c.mean() + 2 * std)
        d.update_fdata('month2stddown', i, prevmonth_df.mid_c.mean() - 2 * std)
        d.update_fdata('month3stdup', i, prevmonth_df.mid_c.mean() + 3 * std)
        d.update_fdata('month3stddown', i, prevmonth_df.mid_c.mean() - 3 * std)

d.df['test'].ffill(inplace=True)

In [ ]:
d.df['test'][d.df['test'].newmonth]

In [ ]:
cp = CandlePlot(d.df['test'], candles=False)
# cp.create_candle_fig()
cp.show_plot(line_traces=['mid_c', 'day1stdup', 'day1stddown', 'daymean'])

In [ ]:
cp = CandlePlot(d.df['test'], candles=False)
# cp.create_candle_fig()
cp.show_plot(line_traces=['mid_c', 'month1stdup', 'month1stddown', 'monthmean'])